# Hourly Data #

In [1]:
# imports
import pandas as pd
import numpy as np
import time

In [2]:
# define data location
#location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
location = '/Users/loki/Documents/Data/Forecasting Project/'

## Data Cleaning & Checking 

In [3]:
# hourly data
use_jan_in = pd.read_excel(location+'Zip_HourlylUsage_2018.01.xlsx')
use_feb_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.02.xlsx')
use_mar_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.03.xlsx')
use_apr_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.04.xlsx')
use_may_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.05.xlsx')
use_jun_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.06.xlsx')
use_jul_in = pd.read_excel(location+'Zip_HourlylUsage_2018.07.xlsx')
use_aug_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.08.xlsx')
use_sep_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.09.xlsx')
use_oct_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.10.xlsx')
use_nov_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.11.xlsx') 
use_dec_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.12.xlsx') 

#weather data
temp_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="TMP")
humid_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="HUM")
wind_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="WSP")
cloud_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="CC")

# other data
customer_in = pd.read_excel(location+'PECO Zip Customer 2018.10.01 v2.xlsx', sheet_name="Account")
ratecode_in = pd.read_excel(location+'Rate Codes for Drexel 9_28_2018.xlsx')


In [4]:
# merge use data
hourly_in = [use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
             use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in]
use = pd.concat(hourly_in)

#hourly = hourly.rename(columns={'DAccountID':'DACCOUNTID', 'DMeterNo':'DMETERNO',
#                              'DAILY_INTERVAL_USAGE':'Use'})

# list(hourly)

In [5]:
# identify unique ids/meterno pairs
def uniqueIDs(dfs, cols):
    """
    Identifies uniques in specified cols and intersects across dfs to return only the overlap
    
    Parameters
    ----------
    dfs : list of pandas data frames with consistent headers
    cols : list of column names to track

    Returns
    ----------
    ids : pandas df with cols columns containing only ids present in all dfs
    """
    
    dfs2 = []
    for df in dfs:
        dfs2.append(df.loc[:, df.columns.isin(cols)].drop_duplicates())
   
    ids = dfs2[0]
    for i in range(1,len(dfs2)):
        ids = pd.merge(ids, dfs2[i], how='inner')
        
    return ids

In [6]:
# identify unique ids/meterno pairs
ids = uniqueIDs(hourly_in,['DACCOUNTID','DMETERNO'])

In [7]:
# clean
use = use.loc[use['UOM'] == 'CCF']
use = use.drop(columns=['UOM'])

customer = customer_in.loc[customer_in['FUELTYPE'] == 'GAS']
customer = customer.drop(columns=['CITY', 'STATE', 'ZIPCODE', 'FUELTYPE', 'COUNTYCODE'])

temp = temp_in.drop(columns=['Avg','HighDB','LowDB','AvgHL','Gas Day Average','HDD-HL','CDD-HL','HDD-24','CDD-24',
                            'Unnamed: 34','Unnamed: 35'])
humid = humid_in.drop(columns=['Avg','Unnamed: 26','Unnamed: 27','Unnamed: 28'])
wind = wind_in.drop(columns=['Avg','Unnamed: 26'])
cloud = cloud_in.drop(columns=['AvgDaytime','Avg'])

# convert to datetime
use['Dt'] =  pd.to_datetime(use['METERREADDATE'])
temp['Dt'] =  pd.to_datetime(temp['Dt'])
humid['Dt'] =  pd.to_datetime(humid['Dt'])
wind['Dt'] =  pd.to_datetime(wind['Dt'])
cloud['Dt'] =  pd.to_datetime(cloud['Dt'])

## Data Restructuring ##

In [8]:
# functions for naming consistency
def decrement(x, startswith, split):
    """
    decrements a passed string of form "demo#" by 1
    
    Parameters
    ----------
    x : string to be decremented
    split : string to split on

    Returns
    ----------
    y : decremented string
    """
    if x.startswith(startswith):
        a,b = x.split(split)
        b = int(b)-1
        y = a + split + str(b)

        return y

    else:
        return x

    
def interval_to_hour(df):
    """
    function for fast rename/relabel during tidying process
    
    Parameters
    ----------
    df : pandas data frame

    Returns
    ----------
    df : data frame with updated column names
    """
    
    df = df.rename(columns=lambda x: decrement(x, "INTERVAL_", "_"))
    df = df.rename(columns=lambda x: x.replace("INTERVAL_", "HR"))
    return df

In [9]:
# rename for consistency
use = interval_to_hour(use)
use = use.drop(columns=['METERREADDATE','HR24'])

In [10]:
# Tidy / Stack data (transform into tall data - one row per customer per hour):
# ref: http://www.jeannicholashould.com/tidy-data-in-python.html
tidy_use = pd.melt(use, 
                   id_vars=['DACCOUNTID','DMETERNO','Dt'],
                   var_name='Hour', value_name='Use')

tidy_temp = pd.melt(temp, 
                    id_vars=['Dt'],
                    var_name='Hour', value_name='Temp')

tidy_humid = pd.melt(temp,
                     id_vars=['Dt'],
                     var_name='Hour', value_name='Humid')

tidy_wind = pd.melt(temp, 
                    id_vars=['Dt'],
                    var_name='Hour', value_name='Wind')

tidy_cloud = pd.melt(temp, 
                     id_vars=['Dt'],
                     var_name='Hour', value_name='Cloud')


In [11]:
# relabel & retype for sorting
tidy_use['Hour'] = tidy_use['Hour'].str.extract('(\d+)').astype(int)
tidy_temp['Hour'] = tidy_temp['Hour'].str.extract('(\d+)').astype(int)
tidy_humid['Hour'] = tidy_humid['Hour'].str.extract('(\d+)').astype(int)
tidy_wind['Hour'] = tidy_wind['Hour'].str.extract('(\d+)').astype(int)
tidy_cloud['Hour'] = tidy_cloud['Hour'].str.extract('(\d+)').astype(int)

# sort by date & time
tidy_use = tidy_use.sort_values(by=["Dt","Hour"])
tidy_temp = tidy_temp.sort_values(by=["Dt","Hour"])
tidy_humid = tidy_humid.sort_values(by=["Dt","Hour"])
tidy_wind = tidy_wind.sort_values(by=["Dt","Hour"])
tidy_cloud = tidy_cloud.sort_values(by=["Dt","Hour"])

In [12]:
def lagDeltas(data, colname, newname, lag):
    """Calculates specified lag of provided column and saves as a new column
     
    Parameters
    ----------
    data : pandas dataframe object to be used calculate lag 0 and lag 1 deltas
    colname : name of column to be lagged
    newname : name of result to be added to pandas dataframe
    lag : number of lags to take 

    Returns
    ----------
    lagData : pandas dataframe with new column
    """
    
    data[newname] = data[colname].shift(lag)
    
    return data

In [13]:
# Find lags of weather variables
# Lag temp
for i in range(1,7):
    temp = lagDeltas(tidy_temp, "Temp", "Temp"+str(i), i)

# Lag humid
for i in range(1,7):
    humid = lagDeltas(tidy_humid, "Humid", "Humid"+str(i), i)
    
# Lag wind
for i in range(1,7):
    wind = lagDeltas(tidy_wind, "Wind", "Wind"+str(i), i)

# Lag cloud
for i in range(1,7):
    cloud = lagDeltas(tidy_cloud, "Cloud", "Cloud"+str(i), i)


In [15]:
# Merge
weather1 = pd.merge(temp, humid, how='inner', on=['Dt', 'Hour'])
weather2 = pd.merge(wind, cloud, how='inner', on=['Dt', 'Hour'])
weather = pd.merge(weather1, weather2, how='inner', on=['Dt', 'Hour'])

hourly = pd.merge(tidy_use, weather, how='inner', on=['Dt', 'Hour'])
# CustIDs || Date | Time | Consumption |||| Weather_variables 

In [16]:
# Add dummy variables for day-of-week and holidays
hourly = hourly.join(pd.get_dummies(hourly['Dt'].dt.weekday_name))

from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
dr = pd.date_range(start='2010-01-01', end='2018-12-31')
holidays = cal.holidays(start=dr.min(), end=dr.max())

hourly = hourly.join(pd.get_dummies(hourly['Dt'].isin(holidays)))
hourly = hourly.rename(columns={True:'Holiday'})
hourly = hourly.drop(columns=[False])


In [17]:
# keep only records with full year of data
hourly = hourly.merge(ids, how='inner', on=['DACCOUNTID','DMETERNO'])

In [20]:
# cross-reference ids with sufficient data against rate code list
# rate_codes = ids.merge(customer, how='inner', on=['DACCOUNTID','DMETERNO'])

# we need to use customer_in instead of customer because customer filters out electric rate codes, some of whom still use gas
rate_codes = ids.merge(customer_in, how='inner', on=['DACCOUNTID','DMETERNO'])

print(len(ids))
print(len(rate_codes))
print(f"There is {100*len(ids)/len(rate_codes)} percent overlap between customer rate code info and gas customer use info")

13806
13877
There is 99.48836203790445 percent overlap between customer rate code info and gas customer use info


In [21]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
hourly.to_pickle(location+'peco_hourly.pkl.zip') 
ids.to_pickle(location+'peco_hourly_ids.pkl.zip')
rate_codes.to_pickle(location+'peco_hourly_rate_codes.pkl.zip')